In [ ]:
import glob
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import mir_eval
import pandas as pd

import IPython.display as ipd

In [ ]:
plt.rcParams["figure.figsize"] = (15,10)

In [ ]:
FS = 44100
CHANNELS = [0, 32, 64, 128]

In [ ]:
candombe_audio_path = '../datasets/candombe/csic.1995_ansina2_04.wav'

In [ ]:
candombe, _ = librosa.load(candombe_audio_path, sr=FS)

In [ ]:
x_df = pd.read_csv(candombe_audio_path.replace('.wav', '.csv'), names=["timestamp", "beat"])
ground_truth = x_df['timestamp'].values

In [ ]:
onset_subbands = librosa.onset.onset_strength_multi(y=candombe, sr=FS, channels=CHANNELS)
times = librosa.times_like(onset_subbands, sr=FS)
# true_beats = ground_truth[(ground_truth >= start) & (ground_truth < end)] - start
onset_frames = []

for i in range(onset_subbands.shape[0]):
    onset_frames.append(librosa.onset.onset_detect(onset_envelope=onset_subbands[i], sr=FS))

# cálculo padrão dos beats
beat_frames = []

for i in range(len(onset_subbands)):
    _, beats = librosa.beat.beat_track(onset_envelope=onset_subbands[i], sr=FS)
    beat_frames.append(beats)

for i in range(len(onset_subbands)):
    print(f"{'*'*10} subband {i} {'*'*10}")
    print(f"fmeasure {mir_eval.beat.f_measure(ground_truth, librosa.frames_to_time(beat_frames[i], FS))}")
    CMLc, CMLt, AMLc, AMLt = mir_eval.beat.continuity(ground_truth, librosa.frames_to_time(beat_frames[i], FS))
    print(f"CMLc = {CMLc}\nCMLt = {CMLt}\nAMLc = {AMLc}\nAMLt = {AMLt}")

In [ ]:
num_plots = onset_subbands.shape[0]
fig, ax = plt.subplots(nrows=num_plots, sharex=True)

for i in range(num_plots):
    ax[i].set_title(f"subband_{i}")
    ax[i].plot(times, onset_subbands[i], alpha=0.6, label=f'subband_{i}')
    ax[i].vlines(times[onset_frames[i]], 0, onset_subbands[i].max(), alpha=0.6, color='r', linestyle='--', label='onsets')
    ax[i].vlines(times[beat_frames[i]], 0, onset_subbands[i].max(), alpha=0.9, color='g', label='detected beat')
    ax[i].vlines(true_beats, 0, onset_subbands[i].max(), color='b', alpha=0.9, label='true beat')
    
fig.legend()

In [ ]:
song = '../datasets/candombe/csic.1995_ansina2_01.wav'

x2, _ = librosa.load(song, mono=True, sr=FS)
x2_df = pd.read_csv(candombe_audio_path.replace('.wav', '.csv'), names=["timestamp", "beat"])
ground_truth2 = x2_df['timestamp'].values

onset_subbands = librosa.onset.onset_strength_multi(y=x2, sr=FS, channels=CHANNELS)
times = librosa.times_like(onset_subbands, sr=FS)
onset_frames = []

for i in range(onset_subbands.shape[0]):
    onset_frames.append(librosa.onset.onset_detect(onset_envelope=onset_subbands[i], sr=FS))

# cálculo padrão dos beats
beat_frames = []

for i in range(len(onset_subbands)):
    _, beats = librosa.beat.beat_track(onset_envelope=onset_subbands[i], sr=FS)
    beat_frames.append(beats)
    
for i in range(len(onset_subbands)):
    print(f"{'*'*10} subband {i} {'*'*10}")
    print(f"fmeasure {mir_eval.beat.f_measure(ground_truth2, librosa.frames_to_time(beat_frames[i], FS))}")
    CMLc, CMLt, AMLc, AMLt = mir_eval.beat.continuity(ground_truth2, librosa.frames_to_time(beat_frames[i], FS))
    print(f"CMLc = {CMLc}\nCMLt = {CMLt}\nAMLc = {AMLc}\nAMLt = {AMLt}")

In [ ]:
num_plots = onset_subbands.shape[0]
fig, ax = plt.subplots(nrows=num_plots, sharex=True)

for i in range(num_plots):
    ax[i].set_title(f"subband_{i}")
    ax[i].plot(times, onset_subbands[i], alpha=0.6, label=f'subband_{i}')
    ax[i].vlines(times[onset_frames[i]], 0, onset_subbands[i].max(), alpha=0.6, color='r', linestyle='--', label='onsets')
    ax[i].vlines(times[beat_frames[i]], 0, onset_subbands[i].max(), alpha=0.9, color='g', label='detected beat')
    ax[i].vlines(true_beats, 0, onset_subbands[i].max(), color='b', alpha=0.9, label='true beat')
    
fig.legend()

In [ ]:
# good performance song
song = '../datasets/candombe/zavala.muniz.2014_52.wav'

x2, _ = librosa.load(song, mono=True, sr=FS)
x2_df = pd.read_csv(candombe_audio_path.replace('.wav', '.csv'), names=["timestamp", "beat"])
ground_truth2 = x2_df['timestamp'].values

onset_subbands = librosa.onset.onset_strength_multi(y=x2[start*FS:end*FS], sr=FS, channels=[0, 4, 8, 64, 128])
times = librosa.times_like(onset_subbands, sr=FS)
true_beats = ground_truth2[(ground_truth2 >= start) & (ground_truth2 < end)] - start
onset_frames = []

for i in range(onset_subbands.shape[0]):
    onset_frames.append(librosa.onset.onset_detect(onset_envelope=onset_subbands[i], sr=FS))

# cálculo padrão dos beats
beat_frames = []

for i in range(len(onset_subbands)):
    _, beats = librosa.beat.beat_track(onset_envelope=onset_subbands[i], sr=FS)
    beat_frames.append(beats)

num_plots = onset_subbands.shape[0]
fig, ax = plt.subplots(nrows=num_plots, sharex=True)

for i in range(num_plots):
    ax[i].set_title(f"subband_{i}")
    ax[i].plot(times, onset_subbands[i], alpha=0.6, label=f'subband_{i}')
    ax[i].vlines(times[onset_frames[i]], 0, onset_subbands[i].max(), alpha=0.6, color='r', linestyle='--', label='onsets')
    ax[i].vlines(times[beat_frames[i]], 0, onset_subbands[i].max(), alpha=0.9, color='g', label='detected beat')
    ax[i].vlines(true_beats, 0, onset_subbands[i].max(), color='b', alpha=0.9, label='true beat')
    
fig.legend()

In [ ]:
for i in range(len(onset_subbands)):
    print(f"{'*'*10} subband {i} {'*'*10}")
    print(f"fmeasure {mir_eval.beat.f_measure(true_beats, librosa.frames_to_time(beat_frames[i], FS))}")
    CMLc, CMLt, AMLc, AMLt = mir_eval.beat.continuity(true_beats, librosa.frames_to_time(beat_frames[i], FS))
    print(f"CMLc = {CMLc}\nCMLt = {CMLt}\nAMLc = {AMLc}\nAMLt = {AMLt}")

# rodando pra todos os áudios e checando se houve ou não diferença significativa no geral

In [ ]:
dataset_result = {}

In [ ]:
file_path = [i[:-4] for i in glob.glob('../datasets/candombe/*.wav')]

for file in file_path:
        print(f"processing {file}")
        x, fs = librosa.load(f"{file}.wav", mono=True, sr=FS)
        x_df = pd.read_csv(f"{file}.csv", names=["timestamp", "beat"])
        ground_truth = x_df['timestamp'].values
        beat_frames = []
        onset_frames = []
        librosa_timestamps = []
        
        onset_subbands = librosa.onset.onset_strength_multi(y=x, sr=FS, channels=[0, 4, 8, 32, 128])
        dataset_result[file] = {}
        
        for i in range(len(onset_subbands)):
            onset_frames.append(librosa.onset.onset_detect(onset_envelope=onset_subbands[i], sr=FS))
            
        for i in range(len(onset_subbands)):
            _, beats = librosa.beat.beat_track(onset_envelope=onset_subbands[i], sr=FS)
            beat_frames.append(beats)
            librosa_timestamp = librosa.frames_to_time(beats, sr=FS)
            librosa_timestamps.append(librosa_timestamp)

        for i in range(len(onset_subbands)):
            cmlc_librosa, cmlt_librosa, amlc_librosa, amlt_librosa = mir_eval.beat.continuity(ground_truth, librosa_timestamps[i])
            
            dataset_result[file][f"subband{i}"] = {
                "ground_truth_beats": ground_truth,
                "librosa_beats": librosa_timestamps[i],
                "f_score_librosa": mir_eval.beat.f_measure(ground_truth, librosa_timestamps[i]),
                "p_score_librosa": mir_eval.beat.p_score(ground_truth, librosa_timestamps[i]),
                "cmlc_librosa": cmlc_librosa,
                "cmlt_librosa": cmlt_librosa,
                "amlc_librosa": amlc_librosa,
                "amlt_librosa": amlt_librosa
            }

    

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict({(i,j): dataset_result[i][j] 
                            for i in dataset_result.keys() 
                            for j in dataset_result[i].keys()},
                            orient='index')

In [ ]:
df

In [ ]:
df.xs('subband0', level=1, drop_level=False).median()

In [ ]:
df.xs('subband1', level=1, drop_level=False).median()

In [ ]:
df.xs('subband2', level=1, drop_level=False).median()

In [ ]:
df.xs('subband3', level=1, drop_level=False).median()

In [ ]:
# ../datasets/candombe/csic.1995_ansina2_04 -> 0.011778563015312132
# ../datasets/candombe/csic.1995_ansina2_01 -> 0.013138686131386862
# ../datasets/candombe/proyecto.1992_pelado_05 -> 0.015300546448087432
# ../datasets/candombe/csic.1995_ansina2_02 -> 0.01866475233309404
# ../datasets/candombe/proyecto.1992_gimenez_06 -> 0.023391812865497075
# ../datasets/candombe/csic.1995_ansina2_03 -> 0.029220779220779224
# ../datasets/candombe/csic.1995_ansina1_03 -> 0.0621301775147929
# ../datasets/candombe/proyecto.1992_pelado_01 -> 0.0743801652892562
# ../datasets/candombe/csic.1995_ansina1_02 -> 0.08591885441527446
# ../datasets/candombe/proyecto.1992_magarinos_02 -> 0.12185929648241205
# ../datasets/candombe/proyecto.1992_gimenez_02 -> 0.19180201082753284
# ../datasets/candombe/csic.1995_ansina1_01 -> 0.21518987341772153
# ../datasets/candombe/zavala.muniz.2014_45 -> 0.2403204272363151
# ../datasets/candombe/csic.1995_cuareim_03 -> 0.29405405405405405
# ../datasets/candombe/zavala.muniz.2014_55 -> 0.33396584440227706
# ../datasets/candombe/zavala.muniz.2014_46 -> 0.3815261044176707
# ../datasets/candombe/zavala.muniz.2014_44 -> 0.41504539559014264
# ../datasets/candombe/zavala.muniz.2014_42 -> 0.4383164005805515
# ../datasets/candombe/zavala.muniz.2014_49 -> 0.4440944881889764
# ../datasets/candombe/zavala.muniz.2014_50 -> 0.4794326241134752
# ../datasets/candombe/proyecto.1992_lobo_06 -> 0.50464807436919
# ../datasets/candombe/zavala.muniz.2014_41 -> 0.525697503671072
# ../datasets/candombe/zavala.muniz.2014_51 -> 0.5799793601651188
# ../datasets/candombe/csic.1995_cuareim_02 -> 0.6242990654205608
# ../datasets/candombe/csic.1995_ansina1_05 -> 0.6396255850234009
# ../datasets/candombe/zavala.muniz.2014_47 -> 0.6533333333333333
# ../datasets/candombe/proyecto.1992_lobo_01 -> 0.6620370370370371
# ../datasets/candombe/zavala.muniz.2014_54 -> 0.6733333333333332
# ../datasets/candombe/zavala.muniz.2014_53 -> 0.6961451247165533
# ../datasets/candombe/csic.1995_cuareim_04 -> 0.6992084432717678
# ../datasets/candombe/csic.1995_ansina1_04 -> 0.7230411171450737
# ../datasets/candombe/zavala.muniz.2014_48 -> 0.7267267267267268
# ../datasets/candombe/csic.1995_cuareim_05 -> 0.7457375833951075
# ../datasets/candombe/csic.1995_cuareim_01 -> 0.7503136762860728
# ../datasets/candombe/zavala.muniz.2014_52 -> 0.7821656050955413


In [ ]:
df.xs('subband0', level=1, drop_level=False).sort_values('f_score_librosa', ascending=True)['f_score_librosa']